In [1]:
from IPython.display import HTML, Markdown, display

def markdown_print(string):
    display(Markdown(string))

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [2]:
def color_negative_red(value):
  """
  Colors elements in a dateframe
  green if positive and red if
  negative. Does not color NaN
  values.
  """

  if value < 0:
    color = 'red'
  elif value > 0:
    color = 'green'
  else:
    color = 'black'

  return 'color: %s' % color

In [3]:
#-------------------------------------------------------------------------------
# Name:        SV Comparison
# Purpose:     Debug merged SV instances
#
# Author:      kristjan.vilgo
#
# Created:     24.03.2019
# Copyright:   (c) kristjan.vilgo 2019
# Licence:     GPL V2
#-------------------------------------------------------------------------------
from RDF_parser import load_all_to_dataframe

from itertools import combinations

import pandas

from urlparse import urlparse

import os
import tempfile

from CGMEStools import *



# Pandas settings
pandas.set_option("display.max_rows", 50)
pandas.set_option("display.max_columns", 120)
pandas.set_option("display.width", 1000)
pandas.set_option('expand_frame_repr', True)
pandas.set_option('display.max_colwidth', -1) # Don't truncate cells



# Define here paths that you want to load. Load only SV files you want to compare, usually CGM SV files
paths = [r"C:\IOPs\IOP150519\RSC_MERGE\20190515T0930Z_1D_BALTICRSC-CE_003.zip",
         r"C:\IOPs\IOP150519\RSC_MERGE\20190515T0930Z_1D_CORESO-CE_002.zip",
         r"C:\IOPs\IOP150519\RSC_MERGE\20190515T0930Z_1D_HANS-CE_004.zip",         
         r"C:\IOPs\IOP150519\RSC_MERGE\20190515T0930Z_1D_TSCNET-CE_003_distributed_slack.zip",
         r"C:\Users\kristjan.vilgo\Downloads\20190304T0000Z_ENTSO-E_BD_001.zip",
         r"C:\IOPs\IOP150519\RSC_MERGE\IGM.zip"
]

data = load_all_to_dataframe(paths)


Loading 20190515T0930Z_1D_BALTICRSC-CE_003/20190514T0930Z_1D_ELES_SSH_001.xml
Loading 20190515T0930Z_1D_BALTICRSC-CE_003/20190514T0930Z_1D_HOPS_SSH_001.xml
Loading 20190515T0930Z_1D_BALTICRSC-CE_003/20190514T0930Z_1D_NOSBIH_SSH_001.xml
Loading 20190515T0930Z_1D_BALTICRSC-CE_003/NEW SV.xml
Loading 20190304T0000Z_ENTSO-E_EQ_BD_001.xml
Loading 20190304T0000Z_ENTSO-E_TP_BD_001.xml
Loading 20190515T0930Z_1D_HANS-hops_SSH_004.xml
Loading 20190515T0930Z_1D_HANS-nosbih_SSH_004.xml
Loading 20190515T0930Z_1D_HANS-CE_SV_004.xml
Loading 20190515T0930Z_1D_HANS-eles_SSH_004.xml
Loading 20190515T0930Z_1D_TSCNET-CE_SV_002.xml
Loading 20190515T0930Z_1D_TSCNET-ELES_SSH_002.xml
Loading 20190515T0930Z_1D_TSCNET-HOPS_SSH_002.xml
Loading 20190515T0930Z_1D_TSCNET-NOSBIH_SSH_001.xml
Loading 20190514T2230Z_HOPS_EQ_001.xml
Loading 20190515T0930Z_1D_ELES_SSH_001.xml
Loading 20190515T0930Z_1D_ELES_TP_001.xml
Loading 20190515T0930Z_1D_HOPS_SSH_001.xml
Loading 20190515T0930Z_1D_HOPS_TP_001.xml
Loading 20190515T0930

In [4]:
# Get all loaded FullModel headers
loaded_instances = data.type_tableview("FullModel")

# Configure the columns to be shown
column_list = [u'Model.modelingAuthoritySet', u'Model.description', u'Model.scenarioTime']

# Get unique profiles present
unique_profile_instances = loaded_instances["Model.profile"].unique().tolist()

profile_name_position = 4

# Display all by profile
for profile_instance in unique_profile_instances:

    # Print profile name    
    markdown_print("# " + profile_instance.split("/")[profile_name_position])
    
    # Display table of instaces of profile
    instances_data = loaded_instances[loaded_instances["Model.profile"] == profile_instance][column_list]
    display(instances_data)
    
    # Display concrete classes statistics
    
    statistics_table = pandas.DataFrame()
    
    for instance_UUID, row in instances_data.sort_values("Model.modelingAuthoritySet").iterrows():
        
        #row["Model.modelingAuthoritySet"] 
        statistics_table[instance_UUID] = statistics_ConcreteClasses(data.query("INSTANCE_ID == '{}'".format(instance_UUID)))
        
    display(statistics_table)    


# Topology

KEY,Model.modelingAuthoritySet,Model.description,Model.scenarioTime
ID,,,
075e898d-46fc-4ee1-bf75-bd74973371fe,http://www.nosbih.ba/OperationalPlanning,Created by Transmission Network Analyzer 2.3,2019-05-15T09:30:00.000Z
0e2b93bb-8be8-4078-b0e3-521503ce0bce,http://eles.si/CGMES/2.4.14,EES SLOVENIJA,2019-05-15T09:30:00Z
7fd25c55-02d5-4e68-8566-9fdf58e08fae,http://hops.hr/CGMES/2.4.15,Croatian forecast data for 15.05.2019 09:30,2019-05-15T09:30:00Z


,0e2b93bb-8be8-4078-b0e3-521503ce0bce,7fd25c55-02d5-4e68-8566-9fdf58e08fae,075e898d-46fc-4ee1-bf75-bd74973371fe
ConnectivityNode,990,NaN,NaN
Terminal,923,1220.0,223.0
TopologicalNode,303,400.0,36.0
FullModel,1,1.0,1.0


# SteadyStateHypothesis

KEY,Model.modelingAuthoritySet,Model.description,Model.scenarioTime
ID,,,
098345fc-d5b9-4551-b934-e10e2e9bbd38,http://hops.hr/CGMES/2.4.15,Croatian forecast data for 15.05.2019 09:30,2019-05-15T09:30:00Z
0bce4977-21b4-4094-a79e-f5ec21c7dd8d,http://www.tscnet.eu/PowerGrid/OperationalPlanning,Created by TSCNET using AMICA 1.3,2019-05-15T09:30:00Z
0f3b5db8-a445-4f47-af76-a81906f3613e,http://eles.si/CGMES/2.4.14,20190515T0930Z_NOSBIH_EQ_000-2019-06-10T15:52:37--Base-Network,2019-05-14T09:30:00Z
133c823f-0619-4812-87e3-c499d407b2f6,http://www.tscnet.eu/PowerGrid/OperationalPlanning,Created by TSCNET using AMICA 1.3,2019-05-15T09:30:00Z
1fcfafda-3b14-4242-b7e6-ed166b86bdb6,http://hops.hr/CGMES/2.4.15,20190515T0930Z_NOSBIH_EQ_000-2019-06-10T15:52:37--Base-Network,2019-05-14T09:30:00Z
20e40fd1-fed9-4788-802d-ef9807b1bf67,http://www.nosbih.ba/OperationalPlanning,Created by Transmission Network Analyzer 2.3,2019-05-15T09:30:00.000Z
8190ee8b-5042-47b7-99fa-4eb08fc061da,http://eles.si/CGMES/2.4.14,NaN,2019-05-15T09:30:00Z
8503e8af-2bbe-4415-91ea-73151ceb8cb0,http://www.tscnet.eu/PowerGrid/OperationalPlanning,Created by TSCNET using AMICA 1.3,2019-05-15T09:30:00Z
8e2c1482-7a7f-4243-ab8d-e67086fb3ce9,http://hops.hr/CGMES/2.4.15,NaN,2019-05-15T09:30:00Z


...

KEY,Model.modelingAuthoritySet,Model.description,Model.scenarioTime
ID,,,
24e8e4f1-2486-475b-b50c-2e7d83559c82,http://www.nosbih.ba/OperationalPlanning,Created by Transmission Network Analyzer 2.3,2019-05-15T09:30:00.000Z
2a46904d-7c80-4655-ace4-b91190a87fa8,http://hops.hr/CGMES/2.4.15,Croatian network model data for 14.05.2019 22:30,2019-05-14T22:30:00Z
b132e996-490e-429b-aa4d-79351edf0c3f,http://eles.si/CGMES/2.4.14,EES SLOVENIJA,2019-05-15T09:30:00Z


,b132e996-490e-429b-aa4d-79351edf0c3f,2a46904d-7c80-4655-ace4-b91190a87fa8,24e8e4f1-2486-475b-b50c-2e7d83559c82
Terminal,3385,1220.0,223.0
Disconnector,1181,NaN,NaN
CurrentLimit,1116,1488.0,62.0
ConnectivityNode,990,NaN,NaN
OperationalLimitSet,636,744.0,98.0
ACLineSegment,240,325.0,54.0
VoltageLevel,180,209.0,36.0
ConformLoad,178,NaN,34.0
LoadResponseCharacteristic,178,NaN,36.0
PowerTransformerEnd,156,94.0,16.0


# EquipmentBoundary

KEY,Model.modelingAuthoritySet,Model.description,Model.scenarioTime
ID,,,
5b71db37-35f6-437b-a2bf-ba5d9188d8a9,http://tscnet.eu/EMF,Official CGM boundary set +//-2 years,2019-03-04T00:00:00.000Z


,5b71db37-35f6-437b-a2bf-ba5d9188d8a9
ConnectivityNode,549
Line,549
BaseVoltage,64
EnergySchedulingType,12
FullModel,1
GeographicalRegion,1
SubGeographicalRegion,1


In [5]:
#loaded_instances #DEBUG

In [6]:
sv_instances = loaded_instances[loaded_instances["Model.profile"] == "http://entsoe.eu/CIM/StateVariables/4/1"]

for UUID, instance in sv_instances.iterrows():
    
    markdown_print("**Merge dependacies - " + str(instance["Model.description"]) + "**")

    connections, identified_objects, graph = get_relations_from(data, UUID, notebook = True)
    display(graph.show(r"{}.html".format(UUID)))

**Merge dependacies - 20190515T0930Z_NOSBIH_EQ_000-2019-06-10T15:52:37--Base-Network**

**Merge dependacies - CREATED BY TRANSMISSION NETWORK ANALYZER 2.3**

**Merge dependacies - 1D Created by Hans using PowerFactory 2019SP2**

**Merge dependacies - Created by TSCNET using AMICA 1.3**

In [7]:
comparison_dict = {"statistics":{}, "data":{}, "report":{"Instances": loaded_instances}}
SV_UUID_list = data.query("VALUE == 'http://entsoe.eu/CIM/StateVariables/4/1'").ID.tolist()

EMF_namelist = []

for SV_UUID in SV_UUID_list:

    instance_data = data.query("INSTANCE_ID == '{}'".format(SV_UUID))
    authority = instance_data.query("KEY == 'Model.modelingAuthoritySet'").VALUE.item()

    try:
        EMF_name = urlparse(authority).netloc

    except:
        print("No modelling authorityset found or invalid url -> {}, using SV UUID".format(authority))
        EMF_name = SV_UUID
        
    if EMF_name == "":
        print ("Not valid url -> " + authority + " , Using raw string")
        EMF_name = authority


    comparison_dict["statistics"]["{}".format(EMF_name)] = instance_data.types_dict()
    comparison_dict["data"]["{}".format(EMF_name)]       = instance_data

    EMF_namelist.append(EMF_name)


statistics = pandas.DataFrame(comparison_dict["statistics"])
comparison_dict["report"]["SvStatistics"] = statistics

statistics.fillna(0).T.style.applymap(color_negative_red).highlight_max(axis=0)

No modelling authorityset found or invalid url -> None, using SV UUID


,FullModel,SvInjection,SvPowerFlow,SvShuntCompensatorSections,SvStatus,SvTapStep,SvVoltage,TopologicalIsland
862de2ce-d190-4245-af80-1c428d98910a,1,0,864,5,0,65,782,271
eles.si,1,0,2130,2,0,60,593,1
rte-france.fr,1,1,804,5,0,65,784,32
www.tscnet.eu,1,0,772,5,743,65,784,1


In [8]:

# Add here parameters to compere and at what index
settings = [dict(index = "SvTapStep.TapChanger",      merge_column = "SvTapStep.position"),
            dict(index = "SvPowerFlow.Terminal",      merge_column = "SvPowerFlow.p"),
            dict(index = "SvPowerFlow.Terminal",      merge_column = "SvPowerFlow.q"),
            dict(index = "SvVoltage.TopologicalNode", merge_column = "SvVoltage.v"),
            dict(index = "SvVoltage.TopologicalNode", merge_column = "SvVoltage.angle")]



# Create all comparison tables
for setting in settings:

    column_name = setting["merge_column"]
    type_name =   setting["merge_column"].split(".")[0]

    comparison_dict[column_name] = {}
    
    comparison_data = pandas.DataFrame()

    for SV_UUID in comparison_dict["data"].keys():

        rename_dict = {column_name:SV_UUID}

        data_view = comparison_dict["data"][SV_UUID].\
                    type_tableview(type_name).\
                    set_index(setting["index"])\
                    [[column_name]].\
                    rename(rename_dict, axis = "columns").\
                    apply(pandas.to_numeric, errors = "ignore")#, drop = False) # use .reset_index() before .set_index() to keep result UUID

        comparison_data = comparison_data.join(data_view, how = "outer")

    # Report all SV combinations
    combinations_iterator = combinations(comparison_dict["data"].keys(), 2) # Create all unique combnations of merged model providers

    for combination in combinations_iterator:

        diff_column_name = "{} - {}".format(combination[0], combination[1])
        comparison_data[diff_column_name] = comparison_data[combination[0]] - comparison_data[combination[1]]

    comparison_dict["report"][column_name] = comparison_data

    # Add statistics

    #columns = [u'<baltic-rsc.eu> - <rte-france.fr>', u'<baltic-rsc.eu> - <nordic-rsc.net>', u'<baltic-rsc.eu> - <tscnet.eu>', u'<baltic-rsc.eu> - <scc-rsci.com>']

    comparison_dict["report"][column_name + "_" + "statistics"] = comparison_dict["report"][column_name][EMF_namelist].describe()

    
    
# Save to excel    
#excel_writer = pandas.ExcelWriter(r"C:\IOPs\IOP150519\RSC_MERGE\SV_comparison.xlsx")

#for report in comparison_dict["report"].keys():
#    comparison_dict["report"][report].to_excel(excel_writer, sheet_name = report)

#excel_writer.save()


# Some info

#print("done")


#print(loaded_profiles[loaded_profiles["Model.profile"]=="http://entsoe.eu/CIM/StateVariables/4/1"])

#print("all data is avaialbel in 'comparison_dict['report']'")
#print(comparison_dict['report'].keys())



In [9]:
statisitcs_columns = ["count", "min", "max", "std"]

statistics_to_report = ['SvTapStep.position_statistics', 
                        'SvPowerFlow.p_statistics',
                        'SvPowerFlow.q_statistics',
                        'SvVoltage.v_statistics']

# Number of differences to show
show_top = 5

for report_name in statistics_to_report:
    markdown_print("# " + report_name)
    display(comparison_dict['report'][report_name].T[statisitcs_columns].round(2).style.applymap(color_negative_red).highlight_min(axis=0))

    diff_comparison = comparison_dict['report'][report_name.split("_")[0]].dropna()
    print ("Number of common points -> " + str(len(diff_comparison)))
    diff = pandas.DataFrame(diff_comparison.drop(columns = EMF_namelist).abs().sum(), columns = ["sum diff"]).round(2).sort_values("sum diff")
    display(diff)
    
    largest_diff = diff.tail(1).index.item()
    largest_diff_value = diff.tail(1)["sum diff"].item()
    
    if float(largest_diff_value) > 0:

        markdown_print("**Max positive diff**")
        show_columns = EMF_namelist + [largest_diff]
        sorted_diff = diff_comparison[show_columns].sort_values(largest_diff,kind= 'mergesort', ascending = False)
        display(sorted_diff.head(show_top))

        from_UUID = sorted_diff.index[0]
        connections, identified_objects, graph = get_relations_from(data, from_UUID, notebook = True)
        display(graph.show(r"{}.html".format(from_UUID)))

        markdown_print("**Max negative diff**")
        display(sorted_diff.tail(show_top))

        from_UUID = sorted_diff.index[-1]
        connections, identified_objects, graph = get_relations_from(data, from_UUID, notebook = True)
        display(graph.show(r"{}.html".format(from_UUID)))
        
    else:
        print("No differences")
        
    

# SvTapStep.position_statistics

,count,min,max,std
eles.si,60,-17,21,9.19
862de2ce-d190-4245-af80-1c428d98910a,65,-17,21,9.25
www.tscnet.eu,65,-17,21,9.25
rte-france.fr,65,-17,21,9.25


Number of common points -> 60


,sum diff
eles.si - rte-france.fr,0.0
eles.si - www.tscnet.eu,0.0
eles.si - 862de2ce-d190-4245-af80-1c428d98910a,0.0
rte-france.fr - www.tscnet.eu,0.0
rte-france.fr - 862de2ce-d190-4245-af80-1c428d98910a,0.0
www.tscnet.eu - 862de2ce-d190-4245-af80-1c428d98910a,0.0


No differences


...

,sum diff
rte-france.fr - www.tscnet.eu,28.78
eles.si - 862de2ce-d190-4245-af80-1c428d98910a,42.97
rte-france.fr - 862de2ce-d190-4245-af80-1c428d98910a,67.47
eles.si - rte-france.fr,68.18
eles.si - www.tscnet.eu,80.96
www.tscnet.eu - 862de2ce-d190-4245-af80-1c428d98910a,82.27


**Max positive diff**

,eles.si,862de2ce-d190-4245-af80-1c428d98910a,www.tscnet.eu,rte-france.fr,www.tscnet.eu - 862de2ce-d190-4245-af80-1c428d98910a
SvVoltage.TopologicalNode,,,,,
b25d3280-2222-4db8-b2ba-1eeed9752723,405.1936,405.200,406.759,406.278,1.559
8403e6cd-e1ba-49a2-ab09-f65b90a396cd,405.5249,405.439,406.832,406.420,1.393
9d25a1f9e5d14d47b6dcde99c4380b40,408.5402,408.325,409.478,409.205,1.153
e268acaf-1e6c-4e33-ac80-e02668b94273,407.9077,407.695,408.848,408.575,1.153
59f5b6afd47a40ab9df63321cb698e29,408.5439,408.326,409.478,409.206,1.152


**Max negative diff**

,eles.si,862de2ce-d190-4245-af80-1c428d98910a,www.tscnet.eu,rte-france.fr,www.tscnet.eu - 862de2ce-d190-4245-af80-1c428d98910a
SvVoltage.TopologicalNode,,,,,
f14e3a40-f5f5-8471-a378-c80a15ae6e0c,111.2513,111.265,110.861,111.301,-0.404
09d0c878-951d-23cd-aeda-c7fd133221e8,110.8915,110.924,110.510,110.959,-0.414
2416a679-626c-7a5d-ac39-ed056b2ba4ef,397.7422,397.769,397.345,397.780,-0.424
00a95f620fa6400b9443781141692f5e,397.6507,397.677,397.252,397.689,-0.425
0f36f2c98d8d4e3d88e3abe68ffe05a4,396.9386,396.965,396.539,396.977,-0.426


In [10]:
voltages = pandas.DataFrame()

markdown_print("**Count of common voltages**")

for name in EMF_namelist:

    voltages[name] = comparison_dict['report'][report_name.split("_")[0]][name].dropna().value_counts()
    
voltages.dropna()

**Count of common voltages**

,eles.si,862de2ce-d190-4245-af80-1c428d98910a,www.tscnet.eu,rte-france.fr
0.0,101,290.0,292.0,292.0


In [11]:
count_column_name = "Number_of_TopologicalNodes"
min_nodes_count = 4 # Number of nodes in topological Island for it to be reported 

markdown_print("**Topological Islands with greater than {} Topological Nodes**".format(min_nodes_count))


# Filter all topological nodes belonging to topological island
TopologicalIsland_TopologicalNodes = data.query("KEY == 'TopologicalIsland.TopologicalNodes'")

# Count number of topological nodes in an island
TopologicalIsland_TopologicalNodes_count = pandas.DataFrame(TopologicalIsland_TopologicalNodes["ID"].value_counts()).reset_index().rename(columns = {"index":"ID", "ID":count_column_name})

# Topological islands names
TopologicalIslands_report = data.query("KEY == 'IdentifiedObject.name'")

# Merge with count
TopologicalIslands_report = TopologicalIslands_report.merge(TopologicalIsland_TopologicalNodes_count, on="ID")

# Merge with insatnce metadata
TopologicalIslands_report = TopologicalIslands_report.merge(sv_instances, left_on="INSTANCE_ID", right_index=True)

# Sort, filter and select columns for display
TopologicalIslands_report.sort_values(count_column_name, ascending = False)\
                        [['ID', 'VALUE', count_column_name, 'Model.modelingAuthoritySet', 'Model.description']]\
                        .query("{} > {}".format(count_column_name, min_nodes_count)).set_index("ID")



**Topological Islands with greater than 4 Topological Nodes**

,VALUE,Number_of_TopologicalNodes,Model.modelingAuthoritySet,Model.description
ID,,,,
fba45f7e-d9dd-4622-bf38-e2eb1e889e7a,TopoIsland 1,492,http://eles.si/CGMES/2.4.14,20190515T0930Z_NOSBIH_EQ_000-2019-06-10T15:52:37--Base-Network
e6c402c2-c7cc-4241-9937-af9032600221,N1,492,http://www.tscnet.eu/PowerGrid/OperationalPlanning,Created by TSCNET using AMICA 1.3
f213bf73-a70a-55f6-be14-23b56b494f62,_f213bf73-a70a-55f6-be14-23b56b494f62,458,http://rte-france.fr/Planning/CGMES/2.4.15,CREATED BY TRANSMISSION NETWORK ANALYZER 2.3
45fd2cc9-8f70-4bf6-9f15-fda1b9555512,Island_001,452,None,1D Created by Hans using PowerFactory 2019SP2
4f9e1b08-f87e-5eb6-a370-87af25710b9a,_4f9e1b08-f87e-5eb6-a370-87af25710b9a,216,http://rte-france.fr/Planning/CGMES/2.4.15,CREATED BY TRANSMISSION NETWORK ANALYZER 2.3


In [12]:
# Get diff of nodes between two islands

#data.query("(ID == 'e6c402c2-c7cc-4241-9937-af9032600221' or ID == 'f213bf73-a70a-55f6-be14-23b56b494f62') and KEY == 'TopologicalIsland.TopologicalNodes'").drop_duplicates(["VALUE"],keep = False).VALUE.tolist()


In [13]:
markdown_print("# Synchronous machine comparison")
display(statistics_GeneratingUnit_types(data).round(1)) 

# Synchronous machine comparison

,VALUE,TOTAL,%
HydroGeneratingUnit,94,166,56.6
ThermalGeneratingUnit,40,166,24.1
WindGeneratingUnit,21,166,12.7
GeneratingUnit,10,166,6.0
NuclearGeneratingUnit,1,166,0.6


In [14]:
# Get Synchronous Machines, Terminals, Generating Units and Regulating controls
Terminals               = data.type_tableview("Terminal")
SynchronousMachines     = data.type_tableview("SynchronousMachine")

# Merge with Terminals
SynchronousMachines = pandas.merge(SynchronousMachines.reset_index(),
                                   Terminals.reset_index(), 
                                   suffixes=('', '_Terminal'),
                                   how = "inner",
                                   left_on = "ID",
                                   right_on = 'Terminal.ConductingEquipment')

# Query for referenced objects
GeneratingUnits    = tableview_by_IDs(data, SynchronousMachines,"RotatingMachine.GeneratingUnit")
RegulatingControls = tableview_by_IDs(data, SynchronousMachines,"RegulatingCondEq.RegulatingControl")

# Add data from referenced objects
SynchronousMachines = pandas.merge(SynchronousMachines,
                                   GeneratingUnits,
                                   left_on = "RotatingMachine.GeneratingUnit",
                                   right_index = True,
                                   how= "left",
                                   suffixes=('', '_GeneratingUnit'))

SynchronousMachines = pandas.merge(SynchronousMachines,
                                   RegulatingControls,
                                   left_on = "RegulatingCondEq.RegulatingControl",
                                   right_index = True,
                                   how= "left",
                                   suffixes=('', '_RegulatingControl'))










In [15]:
# Add powerflow results to Synchronous Machines

sv_instances = loaded_instances[loaded_instances["Model.profile"]=="http://entsoe.eu/CIM/StateVariables/4/1"]

for UUID, row in sv_instances.iterrows():

    # Get some display name
    authority = row['Model.modelingAuthoritySet']

    try:
        EMF_name = urlparse(authority).netloc

    except:
        print("No modelling authorityset found or invalid url -> {}, using SV UUID".format(authority))
        EMF_name = SV_UUID
        
    # Get SV data
    SvVoltages    = data.query("INSTANCE_ID == '{}'".format(UUID)).type_tableview("SvVoltage").add_prefix(EMF_name + "_")
    SvPowerFlows  = data.query("INSTANCE_ID == '{}'".format(UUID)).type_tableview("SvPowerFlow").add_prefix(EMF_name + "_")

    # Merge SV data
    SynchronousMachines = pandas.merge(SynchronousMachines, SvVoltages,   suffixes=('', '_SvVoltage'),   how = "left", left_on = 'Terminal.TopologicalNode', right_on = EMF_name + '_SvVoltage.TopologicalNode')
    SynchronousMachines = pandas.merge(SynchronousMachines, SvPowerFlows, suffixes=('', '_SvPowerFlow'), how = "left", left_on = 'ID_Terminal', right_on = EMF_name + '_SvPowerFlow.Terminal')
    
# Add to report    
comparison_dict["report"]["SynchronousMachines"] = SynchronousMachines

No modelling authorityset found or invalid url -> None, using SV UUID


In [16]:
# Create report on Synchronous Machines powerflow results

parameters_to_compare = ["SvPowerFlow.p", "SvPowerFlow.q", "SvVoltage.v"]

for report_name in parameters_to_compare:

    # Report all SV combinations
    combinations_iterator = combinations(comparison_dict["data"].keys(), 2)

    diff_column_name_list = []
    result_column_list = []

    for name in EMF_namelist: 
        result_column_list.append(name + "_" + report_name)

    for combination in combinations_iterator:

        diff_column_name = "{} - {}_{}".format(combination[0], combination[1], report_name)
        diff_column_name_list.append(diff_column_name
                                    )
        SynchronousMachines[diff_column_name] = pandas.to_numeric(SynchronousMachines[combination[0] + "_" + report_name]) - pandas.to_numeric(SynchronousMachines[combination[1] + "_" + report_name])


    #Show results
    #display(SynchronousMachines)

    markdown_print("# " + report_name)
    #display(SynchronousMachines.T[statisitcs_columns].round(2).style.applymap(color_negative_red).highlight_min(axis=0))

    diff_comparison = SynchronousMachines.set_index("ID")[diff_column_name_list].dropna()
    print ("Number of common points -> " + str(len(diff_comparison)))
    diff = pandas.DataFrame(diff_comparison.abs().sum(), columns = ["sum diff_" + report_name]).round(2).sort_values("sum diff_" + report_name)
    display(diff)

    largest_diff = diff.tail(1).index.item()
    largest_diff_value = diff.tail(1)["sum diff_" + report_name].item()

    if float(largest_diff_value) > 0:

        markdown_print("**Max positive diff**")

        # Columns to be shown in short comparison
        show_columns = ["IdentifiedObject.name", "Type_GeneratingUnit", "RegulatingCondEq.controlEnabled", "ACDCTerminal.connected"] + result_column_list + [largest_diff]

        # Sort output
        sorted_diff = SynchronousMachines.set_index("ID")[show_columns].sort_values(largest_diff,kind= 'mergesort', ascending = False)
        
        # Show top positive diff
        display(sorted_diff.head(show_top))

        # Draw diagram
        #from_UUID = sorted_diff.index[0]
        #connections, identified_objects, graph = get_relations_from(data, from_UUID, notebook = True)
        #display(graph.show(r"{}.html".format(from_UUID)))

        markdown_print("**Max negative diff**")
        
        # Show top negative diff
        display(sorted_diff.tail(show_top))

        #from_UUID = sorted_diff.index[-1]
        #connections, identified_objects, graph = get_relations_from(data, from_UUID, notebook = True)
        #display(graph.show(r"{}.html".format(from_UUID)))

    else:
        print("No differences")

# SvPowerFlow.p

Number of common points -> 123


,sum diff_SvPowerFlow.p
KEY,
rte-france.fr - 862de2ce-d190-4245-af80-1c428d98910a_SvPowerFlow.p,205.61
rte-france.fr - www.tscnet.eu_SvPowerFlow.p,229.82
www.tscnet.eu - 862de2ce-d190-4245-af80-1c428d98910a_SvPowerFlow.p,231.92
eles.si - 862de2ce-d190-4245-af80-1c428d98910a_SvPowerFlow.p,285.78
eles.si - www.tscnet.eu_SvPowerFlow.p,354.12
eles.si - rte-france.fr_SvPowerFlow.p,354.75


**Max positive diff**

KEY,IdentifiedObject.name,Type_GeneratingUnit,RegulatingCondEq.controlEnabled,ACDCTerminal.connected,eles.si_SvPowerFlow.p,862de2ce-d190-4245-af80-1c428d98910a_SvPowerFlow.p,www.tscnet.eu_SvPowerFlow.p,rte-france.fr_SvPowerFlow.p,eles.si - rte-france.fr_SvPowerFlow.p
ID,,,,,,,,,
cbc08994-8e50-4db1-825c-caab1e5bf5ae,TES6-GEN,ThermalGeneratingUnit,true,true,-522.8036,-550.084,-567.194,-566.542,43.7384
f3e3986c-5ff1-4a7e-9662-e346a42023bc,WHECAP2_WH,HydroGeneratingUnit,true,true,-142.8037,-148.1,-140.037,-166.962,24.1583
43356728-2691-5635-a0a2-7ed8f0ec0281,WTETUZ2_WT,GeneratingUnit,true,true,-162.8037,-169.257,-195.062,-183.431,20.6273
92a50064-7327-4dc8-89eb-705ecab35a91,WHEVIS1_WH,HydroGeneratingUnit,true,true,-252.8037,-264.463,-264.982,-270.222,17.4183
c2ae4e20-5e99-7c68-36a2-4c1be3fadcc5,WTSTAN1_WT,GeneratingUnit,true,true,-277.5032,-290.544,-280.511,-293.913,16.4098


...

,sum diff_SvVoltage.v
KEY,
rte-france.fr - www.tscnet.eu_SvVoltage.v,2.15
rte-france.fr - 862de2ce-d190-4245-af80-1c428d98910a_SvVoltage.v,2.42
www.tscnet.eu - 862de2ce-d190-4245-af80-1c428d98910a_SvVoltage.v,3.39
eles.si - 862de2ce-d190-4245-af80-1c428d98910a_SvVoltage.v,5.39
eles.si - rte-france.fr_SvVoltage.v,6.01
eles.si - www.tscnet.eu_SvVoltage.v,7.36


**Max positive diff**

KEY,IdentifiedObject.name,Type_GeneratingUnit,RegulatingCondEq.controlEnabled,ACDCTerminal.connected,eles.si_SvVoltage.v,862de2ce-d190-4245-af80-1c428d98910a_SvVoltage.v,www.tscnet.eu_SvVoltage.v,rte-france.fr_SvVoltage.v,eles.si - www.tscnet.eu_SvVoltage.v
ID,,,,,,,,,
0e69a0e1-1f21-4ffb-8472-c6c7e65949f5,G1,WindGeneratingUnit,false,true,113.2809,113.059,112.719,113.07,0.5619
5c1d36e7-4f68-42e2-85eb-f47737e707e4,G1,WindGeneratingUnit,false,true,114.1065,113.849,113.55,113.859,0.5565
3acc1de7-e4d2-446d-9d59-7eb9c75102aa,ZD6P,WindGeneratingUnit,false,true,118.2998,118.175,118.071,118.125,0.2288
fd1cc3a9-32f7-4bfe-b9ba-e03ba1c271d5,ZD6,WindGeneratingUnit,false,true,118.2998,118.175,118.071,118.125,0.2288
a875cee8-5c66-4620-9661-0cc1fc57d4bc,G1,WindGeneratingUnit,false,true,117.4892,117.43,117.37,117.411,0.1192


**Max negative diff**

KEY,IdentifiedObject.name,Type_GeneratingUnit,RegulatingCondEq.controlEnabled,ACDCTerminal.connected,eles.si_SvVoltage.v,862de2ce-d190-4245-af80-1c428d98910a_SvVoltage.v,www.tscnet.eu_SvVoltage.v,rte-france.fr_SvVoltage.v,eles.si - www.tscnet.eu_SvVoltage.v
ID,,,,,,,,,
c1a05240-9676-41dd-84b6-ae2cf08fcbc0,TG 2,ThermalGeneratingUnit,false,false,NaN,0.,0.0,0,NaN
c5409812-b41e-44ae-83f5-9403b234ea39,G1+G2,ThermalGeneratingUnit,false,false,NaN,0.,0.0,0,NaN
cd5350d0-176a-44c1-841c-63c551aef061,G1,HydroGeneratingUnit,false,false,NaN,0.,0.0,0,NaN
e836346e-e148-4b8f-9eab-cb0ce020b27d,G2,ThermalGeneratingUnit,false,false,NaN,0.,0.0,0,NaN
ff3853cc-0b28-46f8-b31d-27fa6bed0f82,G6,ThermalGeneratingUnit,false,false,NaN,0.,0.0,0,NaN


In [17]:
markdown_print("**Top 5 detailed data**")
display(SynchronousMachines.set_index("ID").sort_values(largest_diff,kind='mergesort', ascending = False).head(show_top))

**Top 5 detailed data**

KEY,Equipment.EquipmentContainer,Equipment.aggregate,IdentifiedObject.description,IdentifiedObject.name,RegulatingCondEq.RegulatingControl,RegulatingCondEq.controlEnabled,RotatingMachine.GeneratingUnit,RotatingMachine.p,RotatingMachine.q,RotatingMachine.ratedPowerFactor,RotatingMachine.ratedS,RotatingMachine.ratedU,SynchronousMachine.InitialReactiveCapabilityCurve,SynchronousMachine.earthing,SynchronousMachine.earthingStarPointR,SynchronousMachine.earthingStarPointX,SynchronousMachine.ikk,SynchronousMachine.maxQ,SynchronousMachine.minQ,SynchronousMachine.mu,SynchronousMachine.operatingMode,SynchronousMachine.qPercent,SynchronousMachine.r,SynchronousMachine.r0,SynchronousMachine.r2,SynchronousMachine.referencePriority,SynchronousMachine.satDirectSubtransX,SynchronousMachine.satDirectSyncX,SynchronousMachine.satDirectTransX,SynchronousMachine.shortCircuitRotorType,SynchronousMachine.type,SynchronousMachine.voltageRegulationRange,SynchronousMachine.x0,SynchronousMachine.x2,Type,ID_Terminal,ACDCTerminal.connected,ACDCTerminal.sequenceNumber,IdentifiedObject.description_Terminal,IdentifiedObject.name_Terminal,Terminal.ConductingEquipment,Terminal.ConnectivityNode,Terminal.TopologicalNode,Terminal.phases,Type_Terminal,Equipment.EquipmentContainer_GeneratingUnit,Equipment.aggregate_GeneratingUnit,GeneratingUnit.genControlSource,GeneratingUnit.initialP,GeneratingUnit.maxOperatingP,GeneratingUnit.minOperatingP,GeneratingUnit.nominalP,GeneratingUnit.normalPF,HydroGeneratingUnit.energyConversionCapability,IdentifiedObject.description_GeneratingUnit,IdentifiedObject.name_GeneratingUnit,Type_GeneratingUnit,WindGeneratingUnit.windGenUnitType,IdentifiedObject.name_RegulatingControl,RegulatingControl.Terminal,RegulatingControl.discrete,RegulatingControl.enabled,RegulatingControl.mode,RegulatingControl.targetDeadband,RegulatingControl.targetValue,RegulatingControl.targetValueUnitMultiplier,Type_RegulatingControl,eles.si_SvVoltage.TopologicalNode,eles.si_SvVoltage.angle,eles.si_SvVoltage.v,eles.si_Type,eles.si_SvPowerFlow.Terminal,eles.si_SvPowerFlow.p,eles.si_SvPowerFlow.q,eles.si_Type_SvPowerFlow,rte-france.fr_SvVoltage.TopologicalNode,rte-france.fr_SvVoltage.angle,rte-france.fr_SvVoltage.v,rte-france.fr_Type,rte-france.fr_SvPowerFlow.Terminal,rte-france.fr_SvPowerFlow.p,rte-france.fr_SvPowerFlow.q,rte-france.fr_Type_SvPowerFlow,862de2ce-d190-4245-af80-1c428d98910a_SvVoltage.TopologicalNode,862de2ce-d190-4245-af80-1c428d98910a_SvVoltage.angle,862de2ce-d190-4245-af80-1c428d98910a_SvVoltage.v,862de2ce-d190-4245-af80-1c428d98910a_Type,862de2ce-d190-4245-af80-1c428d98910a_SvPowerFlow.Terminal,862de2ce-d190-4245-af80-1c428d98910a_SvPowerFlow.p,862de2ce-d190-4245-af80-1c428d98910a_SvPowerFlow.q,862de2ce-d190-4245-af80-1c428d98910a_Type_SvPowerFlow,www.tscnet.eu_SvVoltage.TopologicalNode,www.tscnet.eu_SvVoltage.angle,www.tscnet.eu_SvVoltage.v,www.tscnet.eu_Type,www.tscnet.eu_SvPowerFlow.Terminal,www.tscnet.eu_SvPowerFlow.p,www.tscnet.eu_SvPowerFlow.q,www.tscnet.eu_Type_SvPowerFlow,eles.si - rte-france.fr_SvPowerFlow.p,eles.si - www.tscnet.eu_SvPowerFlow.p,eles.si - 862de2ce-d190-4245-af80-1c428d98910a_SvPowerFlow.p,rte-france.fr - www.tscnet.eu_SvPowerFlow.p,rte-france.fr - 862de2ce-d190-4245-af80-1c428d98910a_SvPowerFlow.p,www.tscnet.eu - 862de2ce-d190-4245-af80-1c428d98910a_SvPowerFlow.p,eles.si - rte-france.fr_SvPowerFlow.q,eles.si - www.tscnet.eu_SvPowerFlow.q,eles.si - 862de2ce-d190-4245-af80-1c428d98910a_SvPowerFlow.q,rte-france.fr - www.tscnet.eu_SvPowerFlow.q,rte-france.fr - 862de2ce-d190-4245-af80-1c428d98910a_SvPowerFlow.q,www.tscnet.eu - 862de2ce-d190-4245-af80-1c428d98910a_SvPowerFlow.q,eles.si - rte-france.fr_SvVoltage.v,eles.si - www.tscnet.eu_SvVoltage.v,eles.si - 862de2ce-d190-4245-af80-1c428d98910a_SvVoltage.v,rte-france.fr - www.tscnet.eu_SvVoltage.v,rte-france.fr - 862de2ce-d190-4245-af80-1c428d98910a_SvVoltage.v,www.tscnet.eu - 862de2ce-d190-4245-af80-1c428d98910a_SvVoltage.v
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [18]:
markdown_print("# Check that SV voltage results are not equal with base voltages")

voltages = pandas.DataFrame() # TODO - Should check that no 0 values in main island

BaseVoltages = data.type_tableview("BaseVoltage")
BaseVoltages["BaseVoltage.nominalVoltage"] = pandas.to_numeric(BaseVoltages["BaseVoltage.nominalVoltage"])
#BaseVoltages.at["11d0b6c9cf1241229fec89dbce938f82", "BaseVoltage.nominalVoltage"] = 402 # Test if logic works, by changing base voltage to mach one voltage result


for name in EMF_namelist:

    voltages[name] = comparison_dict['report'][report_name.split("_")[0]][name].dropna().value_counts()

    
#voltages.dropna()
product = pandas.merge(voltages.reset_index(), BaseVoltages, left_on = "index", right_on = "BaseVoltage.nominalVoltage", how = "inner")

if len(product)== 0:
    print("No voltages are equal to basevoltages")
    
else:
    display(product)

# Check that SV voltage results are not equal with base voltages

No voltages are equal to basevoltages


In [19]:
# Get ACLineSegments with terminals

# Generate classical data views needed to extract relevant data
ACLineSegments          = data.type_tableview("ACLineSegment").reset_index()
Terminals               = data.type_tableview("Terminal").reset_index()

# Lets first filter terminals belonging to ACLineSegments to increase performance
ACLineSegments_terminals = pandas.merge(ACLineSegments[["ID"]].rename(columns={"ID":"ACLineSegment"}),
                                        Terminals,
                                        how="inner",
                                        left_on="ACLineSegment",
                                        right_on="Terminal.ConductingEquipment").drop(columns="ACLineSegment")



# Merge terminals connected to same equipment
branches = pandas.merge(ACLineSegments_terminals, ACLineSegments_terminals, how = "inner", on = "Terminal.ConductingEquipment")

# Filter out termials refering to itself and drop out duplicate entries
branches = branches[(branches["ID_x"] != branches["ID_y"])].drop_duplicates("Terminal.ConductingEquipment") # We want only one entry per ACLineSegment, so we drop the duplicate ones

# Join views to get needed AC line data
ACLineSegments = pandas.merge(ACLineSegments, branches, how = "inner", left_on="ID", right_on = 'Terminal.ConductingEquipment')#.set_index("ID")

#display(ACLineSegments)
#print("Total number of lines {}".format(len(ACLineSegments))) #DEBUG - check that no lines have been missed

In [20]:
# Add powerflow results to ACLine Segments

sv_instances = loaded_instances[loaded_instances["Model.profile"]=="http://entsoe.eu/CIM/StateVariables/4/1"]

for UUID, row in sv_instances.iterrows():

    # Get some display name
    authority = row['Model.modelingAuthoritySet']

    try:
        EMF_name = urlparse(authority).netloc

    except:
        print("No modelling authorityset found or invalid url -> {}, using SV UUID".format(authority))
        EMF_name = SV_UUID
        
    # Get SV data
    SvVoltages    = data.query("INSTANCE_ID == '{}'".format(UUID)).type_tableview("SvVoltage")
    SvPowerFlows  = data.query("INSTANCE_ID == '{}'".format(UUID)).type_tableview("SvPowerFlow")
    
    y_column_prefix = EMF_name + "_y_"
    x_column_prefix = EMF_name + "_x_"    

    # Merge SV data
    ACLineSegments = pandas.merge(ACLineSegments, SvVoltages.add_prefix(y_column_prefix),   suffixes=('', '_SvVoltage_y'),   how = "left", left_on = 'Terminal.TopologicalNode_y', right_on = y_column_prefix + 'SvVoltage.TopologicalNode')
    ACLineSegments = pandas.merge(ACLineSegments, SvPowerFlows.add_prefix(y_column_prefix), suffixes=('', '_SvPowerFlow_y'), how = "left", left_on = 'ID_y', right_on = y_column_prefix + 'SvPowerFlow.Terminal')
    ACLineSegments = pandas.merge(ACLineSegments, SvVoltages.add_prefix(x_column_prefix),   suffixes=('', '_SvVoltage_x'),   how = "left", left_on = 'Terminal.TopologicalNode_x', right_on = x_column_prefix + 'SvVoltage.TopologicalNode')
    ACLineSegments = pandas.merge(ACLineSegments, SvPowerFlows.add_prefix(x_column_prefix), suffixes=('', '_SvPowerFlow_x'), how = "left", left_on = 'ID_x', right_on = x_column_prefix + 'SvPowerFlow.Terminal')

No modelling authorityset found or invalid url -> None, using SV UUID


In [21]:
# Recalculate flows on lines
from numpy import sin, cos, deg2rad, triu


input_data = ACLineSegments

for EMF_name in EMF_namelist:

    # Define input columns
    r_name = "ACLineSegment.r"
    x_name = "ACLineSegment.x"

    u_1_name = '{}_x_SvVoltage.v'.format(EMF_name)
    u_2_name = '{}_y_SvVoltage.v'.format(EMF_name)

    angle_1_name = '{}_x_SvVoltage.angle'.format(EMF_name)
    angle_2_name = '{}_y_SvVoltage.angle'.format(EMF_name)

    # Define output columns

    p_1_name = '{}_x_SvPowerFlow.p_calc'.format(EMF_name)
    p_2_name = '{}_y_SvPowerFlow.p_calc'.format(EMF_name)
    d_p_name = '{}_ACLineSegment.delta_p'.format(EMF_name)

    q_1_name = '{}_x_SvPowerFlow.q_calc'.format(EMF_name)
    q_2_name = '{}_y_SvPowerFlow.q_calc'.format(EMF_name)
    d_q_name = '{}_ACLineSegment.delta_q'.format(EMF_name)



    # Set input variable types
    r = input_data[r_name].astype("float")
    x = input_data[x_name].astype("float")

    u_1 = input_data[u_1_name].astype("float")
    u_2 = input_data[u_2_name].astype("float")

    angle_1 = deg2rad(input_data[angle_1_name].astype("float"))
    angle_2 = deg2rad(input_data[angle_2_name].astype("float"))

    # ACLineSegment formulas

    # Calculate admittance
    Y = 1/(r + x * 1j)
    #Z = r + x * 1j

    # Calcualte complex voltages
    V_1 = u_1 * cos(angle_1) + u_1 * sin(angle_1) * 1j
    V_2 = u_2 * cos(angle_2) + u_2 * sin(angle_2) * 1j

    # Calcualte flows at nodes
    S_1 = V_1 * (Y * V_1 - Y * V_2)
    S_2 = V_2 * (Y * V_2 - Y * V_1)

    p_1 = S_1.real
    p_2 = S_2.real
    d_p = abs(p_1 + p_2)

    q_1 = S_1.imag
    q_2 = S_2.imag
    d_q = abs(q_1 + q_2)

    #display(d_p)

    # Write results
    input_data[p_1_name] = p_1
    input_data[p_2_name] = p_2
    input_data[d_p_name] = d_p

    input_data[q_1_name] = q_1
    input_data[q_2_name] = q_2
    input_data[d_q_name] = d_q


    #S = I * V
    #I = V / R

# Add to report    
comparison_dict["report"]["ACLineSegments"] = ACLineSegments

In [22]:
#ACLineSegments # DEBUG

In [23]:
# Create report on ACLineSegments powerflow results

parameters_to_compare = ["ACLineSegment.delta_p", "ACLineSegment.delta_q"]

report_data = pandas.DataFrame(ACLineSegments[(ACLineSegments["ACDCTerminal.connected_x"] == "true") & (ACLineSegments["ACDCTerminal.connected_y"] == "true")])

show_graphs = False


for report_name in parameters_to_compare:

    # Report all SV combinations
    combinations_iterator = combinations(comparison_dict["data"].keys(), 2)

    diff_column_name_list = []
    result_column_list = []

    for name in EMF_namelist: 
        result_column_list.append(name + "_" + report_name)

    for combination in combinations_iterator:

        diff_column_name = "{} - {}_{}".format(combination[0], combination[1], report_name)
        diff_column_name_list.append(diff_column_name
                                    )
        report_data[diff_column_name] = pandas.to_numeric(report_data[combination[0] + "_" + report_name]) - pandas.to_numeric(report_data[combination[1] + "_" + report_name])



    markdown_print("# " + report_name)


    diff_comparison = report_data.set_index("ID")[diff_column_name_list].dropna()
    print ("Number of common points -> " + str(len(diff_comparison)))
    diff = pandas.DataFrame(diff_comparison.abs().sum(), columns = ["sum diff_" + report_name]).round(2).sort_values("sum diff_" + report_name)
    display(diff)

    largest_diff = diff.tail(1).index.item()
    largest_diff_value = diff.tail(1)["sum diff_" + report_name].item()

    if float(largest_diff_value) > 0:

        markdown_print("**Max positive diff**")

        # Columns to be shown in short comparison
        show_columns = ["IdentifiedObject.name"] + result_column_list + [largest_diff]

        # Sort output
        sorted_diff = report_data.set_index("ID")[show_columns].sort_values(largest_diff,kind= 'mergesort', ascending = False)
        
        # Show top positive diff
        display(sorted_diff.head(show_top))

        
        if show_graphs == True:
            # Draw diagram
            from_UUID = sorted_diff.index[0]
            connections, identified_objects, graph = get_relations_from(data, from_UUID, notebook = True)
            display(graph.show(r"{}.html".format(from_UUID)))

        markdown_print("**Max negative diff**")
        
        # Show top negative diff
        display(sorted_diff.tail(show_top))

        if show_graphs == True:
            
            from_UUID = sorted_diff.index[-1]
            connections, identified_objects, graph = get_relations_from(data, from_UUID, notebook = True)
            display(graph.show(r"{}.html".format(from_UUID)))

    else:
        print("No differences")

# ACLineSegment.delta_p

Number of common points -> 585


,sum diff_ACLineSegment.delta_p
KEY,
eles.si - www.tscnet.eu_ACLineSegment.delta_p,35.86
rte-france.fr - www.tscnet.eu_ACLineSegment.delta_p,70.54
eles.si - rte-france.fr_ACLineSegment.delta_p,70.92
www.tscnet.eu - 862de2ce-d190-4245-af80-1c428d98910a_ACLineSegment.delta_p,266.17
eles.si - 862de2ce-d190-4245-af80-1c428d98910a_ACLineSegment.delta_p,269.81
rte-france.fr - 862de2ce-d190-4245-af80-1c428d98910a_ACLineSegment.delta_p,293.76


**Max positive diff**

KEY,IdentifiedObject.name,eles.si_ACLineSegment.delta_p,862de2ce-d190-4245-af80-1c428d98910a_ACLineSegment.delta_p,www.tscnet.eu_ACLineSegment.delta_p,rte-france.fr_ACLineSegment.delta_p,rte-france.fr - 862de2ce-d190-4245-af80-1c428d98910a_ACLineSegment.delta_p
ID,,,,,,
a76139a1-494d-ab92-8a99-6738b0ceb8af,WTUZLA1_WSAR101_1_400,2.569629,2.539554,3.543995,4.489434,1.949880
d74feb5a-796f-40df-b8e1-21c39906079a,Krk_Melina,1.703349,0.039912,1.652588,1.794189,1.754278
75896e5f-fc0b-66ca-9d31-2854059d4295,WSAR101_WMOST41_1_400,1.961288,1.725734,3.175021,3.309827,1.584094
d9a654e3-ae2f-5c79-62bc-ab479b322070,WTREBI2_WMOST32_1_220,1.190199,0.116542,1.306851,1.435953,1.319411
c17c8bbb-95c0-39c5-27d8-b59af5a2a66f,WTREBI2_WMOST32_2_220,1.186747,0.115838,1.303066,1.431730,1.315892


...

,sum diff_ACLineSegment.delta_q
KEY,
rte-france.fr - www.tscnet.eu_ACLineSegment.delta_q,40.95
eles.si - www.tscnet.eu_ACLineSegment.delta_q,49.78
eles.si - rte-france.fr_ACLineSegment.delta_q,51.43
eles.si - 862de2ce-d190-4245-af80-1c428d98910a_ACLineSegment.delta_q,264.13
www.tscnet.eu - 862de2ce-d190-4245-af80-1c428d98910a_ACLineSegment.delta_q,274.67
rte-france.fr - 862de2ce-d190-4245-af80-1c428d98910a_ACLineSegment.delta_q,288.35


**Max positive diff**

KEY,IdentifiedObject.name,eles.si_ACLineSegment.delta_q,862de2ce-d190-4245-af80-1c428d98910a_ACLineSegment.delta_q,www.tscnet.eu_ACLineSegment.delta_q,rte-france.fr_ACLineSegment.delta_q,rte-france.fr - 862de2ce-d190-4245-af80-1c428d98910a_ACLineSegment.delta_q
ID,,,,,,
bd2d6659-d6c8-4acc-8a73-e4908af3f2a5,DIV-RED,47.987346,3.727846,47.360328,50.610743,46.882897
c4a397f5-8fe1-4ca8-a048-46714935cc12,BER-DIV,49.909310,19.741506,48.034203,50.499854,30.758348
34d972ad-cd20-4408-bd75-9e85892298d1,BER-POD400,28.254322,14.067258,29.197973,29.996288,15.929030
f24b8ec6-8754-4634-807f-f021ebc30692,Zerjavinec_Heviz,15.431181,3.252049,15.343660,15.988327,12.736278
26230963-ff61-4cf9-abc6-8d40e301bdcc,NEK-MAR,12.629683,6.519632,11.652097,12.510427,5.990796


**Max negative diff**

KEY,IdentifiedObject.name,eles.si_ACLineSegment.delta_q,862de2ce-d190-4245-af80-1c428d98910a_ACLineSegment.delta_q,www.tscnet.eu_ACLineSegment.delta_q,rte-france.fr_ACLineSegment.delta_q,rte-france.fr - 862de2ce-d190-4245-af80-1c428d98910a_ACLineSegment.delta_q
ID,,,,,,
0fd5246f-f2d9-6a96-2889-c943de5d68c2,WMOST42_WHECAP2_1_220,1.060154,2.863045,1.375948,1.609925,-1.253120
d74feb5a-796f-40df-b8e1-21c39906079a,Krk_Melina,0.951087,1.921839,0.935365,0.661498,-1.260341
5154fc41-a472-4897-a511-be85e6e3371c,Divaca_Pehlin,0.296096,1.992502,0.355525,0.617158,-1.375344
75896e5f-fc0b-66ca-9d31-2854059d4295,WSAR101_WMOST41_1_400,0.942956,2.268669,0.231590,0.606346,-1.662323
065a9f8b-2651-4635-91c5-35b9bfb74f0d,DIV-PEH,0.152705,2.312838,0.209939,0.517222,-1.795616


In [24]:
#ACLineSegments[(ACLineSegments["ACDCTerminal.connected_x"] == "true") & (ACLineSegments["ACDCTerminal.connected_y"] == "true")]# DEBUG

In [25]:
#for column in ACLineSegments.columns: print("'" + column + "'") # DEBUG

In [26]:
# Sanity check against Neplan export (neplan has p and q on all ends)
#ACLineSegments[['eles.si_x_SvPowerFlow.p', 'eles.si_x_SvPowerFlow.p_calc']]
#display(abs(ACLineSegments['eles.si_x_SvPowerFlow.p'].astype("float") + ACLineSegments['eles.si_y_SvPowerFlow.p'].astype("float")).dropna().sort_values()) # Original Neplan
#display(ACLineSegments['eles.si_ACLineSegment.delta_p'].dropna().sort_values()) # Calculated Neplan

In [27]:
# Sanity check against Neplan export (neplan has p and q on all ends)
pandas.set_option("display.max_rows", 30)
ACLineSegments["diff"] = ACLineSegments['eles.si_x_SvPowerFlow.p'].astype("float") - ACLineSegments['eles.si_x_SvPowerFlow.p_calc'].astype("float")
ACLineSegments[(ACLineSegments["ACDCTerminal.connected_x"] == "true") & (ACLineSegments["ACDCTerminal.connected_y"] == "true")][['ID', 'IdentifiedObject.name', 
                                                                                                                                'eles.si_x_SvPowerFlow.p',
                                                                                                                                'eles.si_x_SvPowerFlow.p_calc',
                                                                                                                                'diff',
                                                                                                                                "ACLineSegment.r", "ACLineSegment.x",
                                                                                                                                'eles.si_x_SvVoltage.angle',
                                                                                                                                'eles.si_x_SvVoltage.v',
                                                                                                                                 'eles.si_y_SvVoltage.angle',
                                                                                                                                 'eles.si_y_SvVoltage.v']].dropna().sort_values("diff")


KEY,ID,IdentifiedObject.name,eles.si_x_SvPowerFlow.p,eles.si_x_SvPowerFlow.p_calc,diff,ACLineSegment.r,ACLineSegment.x,eles.si_x_SvVoltage.angle,eles.si_x_SvVoltage.v,eles.si_y_SvVoltage.angle,eles.si_y_SvVoltage.v
448,bd2d6659-d6c8-4acc-8a73-e4908af3f2a5,DIV-RED,-866.9886,-757.371762,-109.616838,0.8579848,11.51875,-21.68962,416.4991,-18.31549,413.8797
17,065a9f8b-2651-4635-91c5-35b9bfb74f0d,DIV-PEH,-94.72288,-69.279854,-25.443026,1.684884,11.81394,-10.75444,232.0163,-9.67112,235.3636
90,27f95f70-a5d5-4165-8130-e8ea4843d58c,RHE Velebit_Melina,-196.9394,-172.078155,-24.861245,5.875,61.082,-8.104157,415.6658,-4.345704,428.338
515,d74feb5a-796f-40df-b8e1-21c39906079a,Krk_Melina,33.93538,45.826129,-11.890749,3.65,7.28,-8.859027,114.1135,-10.6308,115.9135
518,d85936a1-5105-4799-ac08-91565471ca64,DIV-PAD,-79.71687,-69.103032,-10.613838,0.5932,4.1058,-11.09543,231.5891,-10.75444,232.0163
75,2009587d-fc73-4f47-b590-8611d14468fa,BER-KRS1,-207.5715,-198.479339,-9.092161,2.15698,22.95004,-6.109088,406.7133,-4.484049,409.1495
525,dbd4fb63-69cc-4749-911a-932281bf6625,BER-KRS2,-207.5557,-198.479339,-9.076361,2.15698,22.95004,-6.109088,406.7133,-4.484049,409.1495
513,d5b1f4dc-964c-45f7-9131-b5db8315b412,CIR-ZER,36.03438,44.641056,-8.606676,1.515496,7.819344,-5.711986,232.6391,-6.083651,233.9828
159,4ba09559-379c-41c6-97fc-743f84f4f13c,NEK-ZAG1,194.983,201.956795,-6.973795,0.483,5.2325,-4.484049,409.1495,-4.8428,409.6838
156,49f32cde-8471-4bc8-b629-409c18d2613e,DIV-PIV,-7.099644,-0.150801,-6.948843,4.115979,8.006187,-12.64582,113.4824,-12.6614,114.7731


In [28]:
display(HTML("<style>.container { width:90% !important; }</style>")) # Change report layout width

In [29]:
# Export to excel results

export_path  = r"C:\IOPs\IOP150519\RSC_MERGE\SV_comparison_150519.xlsx" 
excel_writer = pandas.ExcelWriter(export_path)

for report in comparison_dict["report"].keys():
    comparison_dict["report"][report].to_excel(excel_writer, sheet_name = report)

excel_writer.save()